In [3]:
import carla
import time
import datetime
import random
import pandas as pd
import matplotlib.pyplot as plt

noise_level = 0.6

# Connect to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(5.0)
world = client.get_world()

# Get map and spawn points
map = world.get_map()
spawn_points = map.get_spawn_points()
start_point = random.choice(spawn_points)

# Spawn the vehicle
vehicle_bp = world.get_blueprint_library().filter('*firetruck*')[0]
vehicle = world.try_spawn_actor(vehicle_bp, start_point)

if not vehicle:
    print("Failed to spawn vehicle.")
    exit()

print(f"Vehicle spawned at {start_point.location}")

# Disable autopilot (We will manually apply noisy waypoints)
vehicle.set_autopilot(False)

# Attach Collision Sensor
collision_bp = world.get_blueprint_library().find('sensor.other.collision')
collision_sensor = world.spawn_actor(collision_bp, carla.Transform(), attach_to=vehicle)

# Set up CSV File for Results
csv_filename = "waypoint_noise_incidents_0.6.csv"
data_records = []
experiment_duration = 60  # 1 minute in seconds

# Collision detection function
def collision_callback(event):
    print("Collision detected!")
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    data_records.append({
        "Timestamp": timestamp,
        "Collision": "Yes",
        "Waypoint_X": None,
        "Waypoint_Y": None,
        "Waypoint_Z": None,
        "Noisy_Waypoint_X": None,
        "Noisy_Waypoint_Y": None,
        "Noisy_Waypoint_Z": None
    })
    
    # Reset vehicle to a new random spawn point
    reset_location = random.choice(spawn_points)
    vehicle.set_transform(reset_location)
    print("Vehicle reset after collision. Continuing experiment.")

# Cleanup function
def cleanup():
    print("Cleaning up...")
    df = pd.DataFrame(data_records)
    df.to_csv(csv_filename, index=False)
    print(f"Incident data saved to {csv_filename}")
    
    if collision_sensor:
        collision_sensor.destroy()
    if vehicle:
        vehicle.destroy()
    print("Vehicle & sensors destroyed. Exiting program.")

# Noisy Waypoint Function
def noisy_waypoint(original_waypoint, noise):
    noisy_x = original_waypoint.transform.location.x + random.uniform(-noise, noise)
    noisy_y = original_waypoint.transform.location.y + random.uniform(-noise, noise)
    return carla.Transform(carla.Location(x=noisy_x, y=noisy_y, z=original_waypoint.transform.location.z), original_waypoint.transform.rotation)

# Start collision sensor
collision_sensor.listen(collision_callback)

print("Vehicle navigating with noisy waypoints...")

try:
    start_time = time.time()
    while time.time() - start_time < experiment_duration:
        current_location = vehicle.get_location()
        next_waypoint_options = map.get_waypoint(current_location).next(5.0)

        if not next_waypoint_options:
            print("No further waypoints found! Resetting vehicle...")
            vehicle.set_transform(random.choice(spawn_points))
            continue
        
        next_waypoint = next_waypoint_options[0]
        noisy_transform = noisy_waypoint(next_waypoint, noise_level)
        vehicle.set_transform(noisy_transform)

        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        data_records.append({
            "Timestamp": timestamp,
            "Collision": "No",
            "Waypoint_X": next_waypoint.transform.location.x,
            "Waypoint_Y": next_waypoint.transform.location.y,
            "Waypoint_Z": next_waypoint.transform.location.z,
            "Noisy_Waypoint_X": noisy_transform.location.x,
            "Noisy_Waypoint_Y": noisy_transform.location.y,
            "Noisy_Waypoint_Z": noisy_transform.location.z
        })

        print(f"Noisy Waypoint Applied: {noisy_transform.location}")
        time.sleep(0.5)

except KeyboardInterrupt:
    print("\nManual stop detected! Cleaning up before exiting...")
    cleanup()

cleanup()


Vehicle spawned at Location(x=-6.554498, y=137.225952, z=0.600000)
Vehicle navigating with noisy waypoints...
Noisy Waypoint Applied: Location(x=-1.427629, y=137.477859, z=0.000000)
Noisy Waypoint Applied: Location(x=3.150303, y=137.071060, z=0.000000)
Noisy Waypoint Applied: Location(x=8.219851, y=136.891022, z=0.000000)
Noisy Waypoint Applied: Location(x=13.685582, y=137.223557, z=0.000000)
Noisy Waypoint Applied: Location(x=18.147062, y=137.578079, z=0.000000)
Noisy Waypoint Applied: Location(x=22.690159, y=136.893005, z=0.000000)
Noisy Waypoint Applied: Location(x=27.952311, y=137.094833, z=0.000000)
Noisy Waypoint Applied: Location(x=33.522366, y=138.010590, z=0.000000)
Noisy Waypoint Applied: Location(x=39.096657, y=137.371811, z=0.000000)
Noisy Waypoint Applied: Location(x=43.907185, y=137.806000, z=0.000000)
Noisy Waypoint Applied: Location(x=48.384125, y=137.585449, z=0.000000)
Noisy Waypoint Applied: Location(x=53.772705, y=137.928085, z=0.000000)
Noisy Waypoint Applied: Loca